#### Transfer data from from the csv file to mysql database
#### database name:plantperformanceData
#### table name:DOWNTIME_GENERAL_DATA

In [35]:
import pandas as pd
import mysql.connector
from mysql.connector import errorcode
from datetime import datetime

In [36]:
db_config = {
    'user': 'root',
    'password': 'oyediran',
    'host': 'localhost',
    'database': 'plantperformanceData',
}


In [45]:
# CSV file path
csv_file_path = "Main_Downtime_data-Modified.csv"
# Read CSV file into a pandas DataFrame
data = pd.read_csv(csv_file_path)
data_1 = data.fillna('')

In [46]:
data_1.head(10)

,DowntimeID,Production_Date,Location,Downtime_Category,ProcessArea,Downtime,Downtime_Duration
0,1,2023-10-02 0:00,Plant A,Planned Downtime,Planned,Coffee Break #1,15
1,2,2023-10-02 0:00,Plant A,Planned Downtime,Planned,Coffee Break #2,15
2,3,2023-10-02 0:00,Plant A,Planned Downtime,Planned,Lunch Break,30
3,4,2023-10-02 0:00,Plant A,Planned Downtime,Planned,Morning Meeting,15
4,5,2023-10-02 0:00,Plant A,Planned Downtime,Planned,Safety Updates,525
5,6,2023-10-03 0:00,Plant A,Planned Downtime,Planned,Coffee Break #1,15
6,7,2023-10-03 0:00,Plant A,Planned Downtime,Planned,Coffee Break #2,15
7,8,2023-10-03 0:00,Plant A,Planned Downtime,Planned,Lunch Break,30
8,9,2023-10-03 0:00,Plant A,Planned Downtime,Planned,Morning Meeting,15
9,10,2023-10-03 0:00,Plant A,Planned Downtime,Planned,Safety Updates,525


In [47]:
# Connect to MySQL
try:
    cnx = mysql.connector.connect(**db_config)
    cursor = cnx.cursor()

    # Insert data into existing MySQL table
    for index, row in data_1.iterrows():
        insert_query = """
        INSERT INTO DOWNTIME_GENERAL_DATA (
            DowntimeID, Production_Date, Location, 
            Downtime_Category, ProcessArea, Downtime, Downtime_Duration
        ) VALUES (%s, %s, %s, %s, %s, %s, %s);
        """
        
        duration_value = pd.to_numeric(row['Downtime_Duration'], errors='coerce')
        data = (
            int(row['DowntimeID']),
            str(row['Production_Date']),
            str(row['Location']),
            str(row['Downtime_Category']),
            str(row['ProcessArea']),
            str(row['Downtime']),
            int(duration_value)
        
        )
        cursor.execute(insert_query, data)

    # Commit changes
    cnx.commit()

except mysql.connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Something is wrong with your user name or password")
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("Database does not exist")
    else:
        print(err)

finally:
    # Close connections
    if 'cnx' in locals() and cnx.is_connected():
        cursor.close()
        cnx.close()


In [62]:
# CSV file path
csv_file_path = "Daily_tonnage_Modified.csv"
# Read CSV file into a pandas DataFrame
data_tonnage = pd.read_csv(csv_file_path)
data_tonnage = data_tonnage.fillna('')

In [63]:
data_tonnage.head(5)

,ProductionID,Production_Date,Location,Tonnage_Produced,Planned_Operating_Time,Plant_Capacity
0,1,2022-01-05 0:00,Plant A,222.00,600,72
1,2,2022-01-09 0:00,Plant A,3.51,600,72
2,3,2022-01-10 0:00,Plant A,44.00,600,72
3,4,2022-01-14 0:00,Plant A,65.20,600,72
4,5,2022-01-16 0:00,Plant A,289.48,600,72


In [64]:
# Connect to MySQL
try:
    cnx = mysql.connector.connect(**db_config)
    cursor = cnx.cursor()

    # Insert data into the Production_data MySQL table
    for index, row in data_tonnage.iterrows():
        insert_query_production = """
        INSERT INTO Production_data (
            ProductionID, Production_Date, Location, 
            Tonnage_Produced, Planned_Operating_Time, Plant_Capacity
        ) VALUES (%s, %s, %s, %s, %s, %s);
        """
        data_production = (
            int(row['ProductionID']),
            str(row['Production_Date']),
            str(row['Location']),
            float(row['Tonnage_Produced']),
            int(row['Planned_Operating_Time']),
            int(row['Plant_Capacity'])
        )
        cursor.execute(insert_query_production, data_production)

    # Commit changes
    cnx.commit()

    # Print a success message
    print("Data transfer to Production_data table successful!")

except mysql.connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Something is wrong with your user name or password")
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("Database does not exist")
    else:
        print(err)

finally:
    # Close connections
    if 'cnx' in locals() and cnx.is_connected():
        cursor.close()
        cnx.close()

Data transfer to Production_data table successful!


In [56]:
data_tonnage.columns

Index(['ProductionID', 'Production_Date', 'Location', ' Tonnage_Produced',
       'Planned_Operating_Time ', 'Plant_Capacity'],
      dtype='object')